# Evaluation

In [53]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
import pandas as pd
import numpy as np
from src import preprocess as prep
from src.evaluation import pick_test_segments, generate_test_segments, evaluate_segments
from src.Video import Video
from src.Segment import Segment
from src import search
import random
import cv2

## Parameters

In [63]:
NUM_VIDEOS = 20
GRID_SIZE = 2
BINS = [180, 180]
HIST_FRAME_SKIP = 20
REFRESH = True

# vergeet gebruikte params soms dus print ze maar afentoe
def printParams():
    print('Num. Vid {} - Grid {} - Bins {} - Skip {}'.format(NUM_VIDEOS, GRID_SIZE, BINS, HIST_FRAME_SKIP))

## Load training set / generate test set

In [ ]:
printParams()
training_set = prep.load_training_set(range(1, NUM_VIDEOS+1), GRID_SIZE, BINS, HIST_FRAME_SKIP, force_refresh=REFRESH)

Num. Vid 20 - Grid 2 - Bins [180, 180] - Skip 20
Loading / processing dataset...
processing 00013

In [61]:
# Set of 100 custom fragments with duration 20sec
test_set, labels = generate_test_segments(training_set, n=100, duration=20)

In [62]:
# Print statistics
print("TRAINING SET:")
print("Num. videos:   {:d}".format( len(training_set)) )
print("Num. segments: {:d}".format( np.sum([len(video.segments) for video in training_set])) )
print("Duration:      {:,.1f} s".format( np.sum([np.sum([segment.duration() for segment in video.segments]) for video in training_set])) )
print("Num frames:      {:d}".format( np.sum([np.sum([segment.num_frames() for segment in video.segments]) for video in training_set])) )
print("Num histograms:      {:d}".format( np.sum([np.sum([len(segment.histograms) for segment in video.segments]) for video in training_set])) )

TRAINING SET:
Num. videos:   20
Num. segments: 2198
Duration:      7,556.4 s
Num frames:      212462
Num histograms:      2198


In [46]:
# Print statistics
print("TEST SET:")
print("Size: {:d}".format( len(test_set) ))

TEST SET:
Size: 100


<br>

## Run model on test set

In [47]:
for method in [cv2.HISTCMP_CORREL, cv2.HISTCMP_CHISQR, cv2.cv2.HISTCMP_INTERSECT,
               cv2.HISTCMP_BHATTACHARYYA, cv2.HISTCMP_CHISQR_ALT, cv2.HISTCMP_KL_DIV]:
    print('{}'.format(method))
    %timeit -n 10 search.findFrame(test_set[0][0], training_set, method)

for ch in [[0], [1], [0, 1]]:
    print('{}'.format(ch))
    %timeit -n 10 search.findFrame(test_set[0][0], training_set, cv2.HISTCMP_CORREL, channels=ch)


0
10.6 ms ± 643 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
1
10.5 ms ± 550 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
2
10.6 ms ± 622 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
3
10.4 ms ± 213 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
4
10.3 ms ± 316 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
5
10.4 ms ± 230 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
[0]
6.31 ms ± 347 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
[1]
6.36 ms ± 370 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
[0, 1]
10.9 ms ± 480 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [49]:
results = []

for i, histogram in enumerate(test_set):
    print('\rSearching segment {}/{}'.format(i+1, len(test_set)), end='', flush=True)
    
#     results.append(search.find(histogram, training_set, cv2.HISTCMP_INTERSECT))
    results.append(search.findFrame(histogram[0], training_set, 2, cv2.HISTCMP_CHISQR_ALT))

Searching segment 49/100

IndexError: list index out of range

## Evaluate performance

In [52]:
evaluate_segments(results, labels)

Segment evaluation:
Correct: 0
Wrong:   48
Total:   48
TPR:     0.0%
